In [10]:
import math
import tensorflow as tf
import numpy as np
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)

#parameters
seq_length =6
train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#data_preprocessing
xy= xy[:,:27]
test_set = xy[train_size:]
a = xy[:,:2]
b = xy[:,2:]
data_max = np.max(b,1)
data_min = np.min(b,1)
b= MinMaxScaler(b)
xy = np.hstack((a,b))
train_set = xy[:train_size]


#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

# build data set for learing
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#make train_set, validation_set, test_set 
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

saver = tf.train.import_meta_graph("/Users/yeseo/ML_with_Taxidata/code/for_solution/model/my_model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"/Users/yeseo/ML_with_Taxidata/code/for_solution/model/my_model.ckpt")
    
    graph = tf.get_default_graph()
    X = graph.get_tensor_by_name("X:0")
    Y = graph.get_tensor_by_name("Y:0")

    Y_pred = graph.get_tensor_by_name("rnn/fully_connected/BiasAdd:0")
        
    #7th value from initial data
    test_predict = sess.run(Y_pred, feed_dict = {X:[testX[0,:,:]]})
    
    #initial data shape = 6,27
    initial_data = testX[0,:,:]
    
    
    #day&time for 7th data
    if initial_data[5,1] != 47:
        day_time = np.array([[initial_data[5,0],(initial_data[5,1]+1)]])
    else:
        if initial_data[5,0] == 7:
            day_time = np.array([[1,0]])
        else:
            day_time = np.array([[(initial_data[5,0]+1),0]])
    
    time_array = day_time
    
    #make 7th data
    seventh_data = np.hstack((day_time,test_predict))
    
    #1~5 data + 7th data
    new_testX = np.vstack((initial_data[1:,:],seventh_data))
    
    #select range to predict
    for j in range(48):
        #8th value from second data
        temp = sess.run(Y_pred,feed_dict = {X:[new_testX]})
        
        test_predict = np.vstack((test_predict,temp))
        
        if new_testX[5,1] != 47:
            day_time = np.array([[new_testX[5,0],(new_testX[5,1]+1)]])
        else:
            if new_testX[5,0] == 7:
                day_time = np.array([[1,0]])
            else:
                day_time = np.array([[(new_testX[5,0]+1),0]])
        time_array = np.vstack((time_array,day_time))
        seventh_data = np.hstack((day_time,test_predict[[-1],:]))
        new_testX = np.vstack((new_testX[1:,:],seventh_data))

INFO:tensorflow:Restoring parameters from /Users/yeseo/ML_with_Taxidata/code/for_solution/model/my_model.ckpt


In [11]:
test_predict

array([[0.6349317 , 0.6037711 , 0.6693879 , ..., 0.6590887 , 0.5397216 ,
        0.44956884],
       [0.59985167, 0.5762084 , 0.6557769 , ..., 0.63146406, 0.54426557,
        0.464895  ],
       [0.56971115, 0.54918826, 0.64933175, ..., 0.6102208 , 0.5545748 ,
        0.48613024],
       ...,
       [0.6168003 , 0.59304154, 0.66025305, ..., 0.644546  , 0.545816  ,
        0.4612351 ],
       [0.6009035 , 0.57711315, 0.6616614 , ..., 0.6359661 , 0.5605833 ,
        0.4836312 ],
       [0.5821243 , 0.5581138 , 0.66530627, ..., 0.62706167, 0.5832517 ,
        0.5162222 ]], dtype=float32)

In [12]:
# make data to original
def ReturnToOriginal(data,data_max,data_min):
    data_max = data_max+(1e-7)
    for i in range(0,len(data)):
        data[i,:]= (data[i,:]*data_max[i])-(data[i,:]-1)*data_min[i]
    return data

test_predict_to_original = ReturnToOriginal(test_predict,data_max[train_size+seq_length:],data_min[train_size+seq_length:]

SyntaxError: unexpected EOF while parsing (<ipython-input-12-3343b27e821b>, line 8)